In [12]:
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [13]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, classification_report, confusion_matrix
import plotly.express as px
import matplotlib.pyplot as plt

In [14]:
def get_train_data():
    data_train = pd.read_csv('../Datasets/Fashion MNIST/fashion-mnist_train.csv')
    X_train = data_train.drop(['label'], axis=1).to_numpy()
    X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))

    y_train = data_train['label'].to_numpy()

    # One-hot encode the labels
    encoder = OneHotEncoder(sparse=False)
    y_train_onehot = encoder.fit_transform(y_train.reshape(-1, 1))

    return X_train, y_train_onehot

In [15]:
def print_accuracy(model, X_test, y_test):
    predictions = model.predict(X_test)
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = np.argmax(y_test, axis=1)

    accuracy = np.mean(predicted_labels == true_labels)
    print(f"Final Accuracy: {accuracy * 100:.2f}%")
    return true_labels, predicted_labels, predictions

In [16]:
class CNN:
    def __init__(self, input_size, num_classes, count_convolutional_layer):
        self.input_size = input_size
        self.num_classes = num_classes  # Update the initialization
        self.count_convolutional_layer = count_convolutional_layer
        self.weight_count_convolutional = self.get_weight_count_convolutional()
        self.flatten_weight = self.get_flatten_weight()

    def get_flatten_weight(self):
        flatten_weight = []
        for i in range(self.num_classes):
            mask = []
            for j in range(49):
                mask.append(random.uniform(0, 1))
            flatten_weight.append(mask)
        return np.array(flatten_weight)

    def get_weight_count_convolutional(self):
        return np.random.uniform(0, 1, size=(3, 3))

    def flatten(self, input_data):
        return input_data.reshape(-1)

    def convolve2d(self, input_data):
        channels = 1  # Default value for single channel
        if len(input_data.shape) == 2:
            height, width = input_data.shape
            input_data = input_data.reshape((height, width, 1))
        elif len(input_data.shape) == 3:
            height, width, channels = input_data.shape
        else:
            raise ValueError("Input data should be either 2D or 3D")

        mask = np.zeros((height + 2, width + 2, channels))
        mask[1:height + 1, 1:width + 1, :] = input_data
        output_data = []
        for i in range(1, height + 1):
            s_array = []
            for j in range(1, width + 1):
                summ = 0
                for ii in range(-1, 2):
                    for jj in range(-1, 2):
                        summ += np.sum(mask[i + ii, j + jj, :] * self.weight_count_convolutional[ii + 1][jj + 1])
                s_array.append(summ)
            output_data.append(s_array)
        return np.array(output_data)

    def max_pooling(self, data):
        leng = len(data)
        i = 0
        j = 0
        result = []
        while i < leng:
            result_str = []
            while j < leng:
                result_str.append(max([data[i][j], data[i][j + 1], data[i + 1][j], data[i + 1][j + 1]]))
                j += 2
            i += 2
            j = 0
            result.append(result_str)
        return np.array(result)

    def fully_connected(self, flattened_output):
        return np.dot(self.flatten_weight, flattened_output)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

    def train(self, X, y, epoch, learning_rate):
        for eph in range(epoch):
            for index_training_example, training_example in enumerate(X):
                training_example = np.array(training_example)
                conv1_output = self.convolve2d(training_example)
                pool1_output = self.max_pooling(conv1_output)

                conv2_output = self.convolve2d(pool1_output)
                pool21_output = self.max_pooling(conv2_output)

                flattened_output = self.flatten(pool21_output)
                fully_connected = self.fully_connected(flattened_output)
                output = self.sigmoid(fully_connected)

                # Backward pass
                loss_gradient = output - y[index_training_example]
                fc_output_gradient = np.dot(self.flatten_weight.T, loss_gradient)
                fc_output_gradient[fc_output_gradient < 0] = 0

                # Update weights
                self.flatten_weight -= learning_rate * np.outer(loss_gradient, flattened_output)

                # Optionally, update convolutional layer weights here

                # Print or log the loss
                if index_training_example % 1000 == 0:
                    current_loss = np.mean(
                        -y_train * np.log(output + 1e-10) - (1 - y_train) * np.log(1 - output + 1e-10))
                    print(f'Epoch {eph}, Example {index_training_example}, Loss: {current_loss}')
    def predict(self, X):
        predictions = []
        for example in X:
            conv1_output = self.convolve2d(example)
            pool1_output = self.max_pooling(conv1_output)

            conv2_output = self.convolve2d(pool1_output)
            pool2_output = self.max_pooling(conv2_output)

            flattened_output = self.flatten(pool2_output)
            fully_connected = self.fully_connected(flattened_output)
            output = self.sigmoid(fully_connected)
            predictions.append(output)

        return np.array(predictions)



In [17]:
if __name__ == '__main__':
    X_train, y_train = get_train_data()

    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.06, random_state=42)

    simple_conv_net = CNN(
        input_size=X_train[0].shape[0],
        num_classes=y_train.shape[1],  # Update the number of classes based on one-hot encoding shape
        count_convolutional_layer=1)

D:\PycharmProjects\Design_and_training_neural_networks\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning:

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.



In [ ]:
simple_conv_net.train(X=X_train, y=y_train, epoch=3, learning_rate=0.01)

Epoch 0, Example 0, Loss: 20.723265836936413
Epoch 0, Example 1000, Loss: 5.984108379193636
Epoch 0, Example 2000, Loss: 4.146286206380334
Epoch 0, Example 3000, Loss: 2.302585092904046
Epoch 0, Example 4000, Loss: 4.143510039956147
Epoch 0, Example 5000, Loss: 4.143183432141535
Epoch 0, Example 6000, Loss: 4.147347681777819
Epoch 0, Example 7000, Loss: 4.143183432141535
Epoch 0, Example 8000, Loss: 4.143183432141535
Epoch 0, Example 9000, Loss: 4.143510039956147


In [ ]:
def metrics_printing(y_true, y_test, y_pred, predictions):
    print(f'Mean Squared Error: {mean_squared_error(y_true, y_pred)}')
    print(f'R-squared: {r2_score(y_true, y_pred)}')
    print(f'Mean Absolute Error: {mean_absolute_error(y_true, y_pred)}')
    print(classification_report(y_true, y_pred))

In [ ]:
true_labels, predicted_labels, predictions = print_accuracy(simple_conv_net, X_test, y_test)

In [ ]:
metrics_printing(true_labels,y_test, predicted_labels, predictions)

In [ ]:
plt.rcParams['figure.figsize'] = (10, 10)
fig = px.imshow(confusion_matrix(true_labels, predicted_labels), text_auto=True)
fig.update_layout(xaxis_title='Цель', yaxis_title='Прогноз')